In [56]:
import sort_RAFT_table as sRt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit
import itertools
from tqdm import tqdm

In [57]:
# ToDo:
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, Reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# rectify/balance the table
sRt.df["t6h-conversion"] = np.nan
sRt.df["t10h-conversion"] = np.nan

# get all conversion headers, sort them by the hours
conversion_list = []
for column in sRt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)

conversion_list.sort(key=lambda x: int(re.findall(r"\d+", x)[0]))

# get all the Mn and Mw headers
Mn_list = []
Mw_list = []
for column in sRt.df.columns:
    if "Mn" in column:
        Mn_list.append(column)
    if "Mw" in column:
        Mw_list.append(column)

print(conversion_list, Mn_list, Mw_list, sep="\n")

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't6h-conversion', 't8h-conversion', 't10h-conversion', 't15h-conversion']
['t0h-Mn', 't1h-Mn', 't2h-Mn', 't4h-Mn', 't6h-Mn', 't8h-Mn', 't10h-Mn', 't15h-Mn']
['t0h-Mw', 't1h-Mw', 't2h-Mw', 't4h-Mw', 't6h-Mw', 't8h-Mw', 't10h-Mw', 't15h-Mw']


In [58]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))
hours_list.sort()

exact_times = pd.read_excel(sRt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors": # that applies to reactor 15 and the closing of reactors
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else: # that applies to all reactors which are described per row in pairs like "3+4"
        reactor_nr_s = (row[1]["Reactor"]).split("+")
        for reactor_nr in reactor_nr_s:
            row[1]["Reactor"] = reactor_nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

ext_time_corr_df

The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,6,8,10,15
Reactor,,,,,,,,
1,0,1.005556,2.011111,4.018056,6.008333,8.016667,10.011111,15.016667
2,0,1.005556,2.011111,4.018056,6.008333,8.016667,10.011111,15.016667
3,0,1.073611,2.068056,4.077778,6.063889,8.090278,10.084722,15.098611
4,0,1.073611,2.068056,4.077778,6.063889,8.090278,10.084722,15.098611
5,0,1.131944,2.125000,4.143056,6.120833,8.180556,10.158333,15.200000
6,0,1.131944,2.125000,4.143056,6.120833,8.180556,10.158333,15.200000
7,0,1.191667,2.184722,4.205556,6.180556,8.256944,10.233333,15.291667
8,0,1.191667,2.184722,4.205556,6.180556,8.256944,10.233333,15.291667
9,0,1.251389,2.241667,4.273611,6.237500,8.338889,10.306944,15.381944


In [59]:
# replace the "sample determiner" with columns describing for the experiment number (next cell) and the actual reagents that made out the "determiner"
reaction_descriptors_dict = {}
abbreviation_keys = pd.read_excel(sRt.INPUT_FILE_PATH, sheet_name="Legend for Abbreviations")
abbreviation_keys.dropna(inplace=True)
for row in abbreviation_keys.itertuples():
    reaction_descriptors_dict[str(row.Symbol)] = row.Name

reaction_descriptors_dict

{'1': 'Styrene',
 '2': '4-Chlorostyrene',
 '3': '4-Bromostyrene',
 '4': '4-Methylstyrene',
 '5': '4-Methoxystyrene',
 '16': '4-tert-Butylstyrene',
 '6': 'Methyl methacrylate',
 '7': 'Butyl methacrylate',
 '8': 'Lauryl methacrylate',
 '9': '(2-Dimethylaminoethyl)methacrylate',
 '10': 'Benzyl methacrylate',
 '11': 'Methyl acrylate ',
 '12': 'Butyl acrylate',
 '13': 'Lauryl acrylate',
 '14': 'Dimethylaminoethylacrylate',
 '15': 'Benzyl acrylate',
 'A': '2-Cyan-2-propylbenzodithioat',
 'B': '4-Cyano-4-(phenylcarbonothioylthio)pentanoic acid',
 'C': '2-Phenyl-2-propyl benzodithioate',
 'D': '2-Cyano-2-propyl dodecyl trithiocarbonate',
 'E': '2-(Dodecylthiocarbonothioylthio)-2-methylpropionic acid',
 'F': 'Cyanomethyl dodecyl trithiocarbonate',
 'G': 'Benzyl 1H-pyrrole-1-carbodithioate',
 'H': '2-Cyanopropan-2-yl N-methyl-N-(pyridin-4-yl)carbamodithioate',
 'I': 'S,S-Dibenzyl trithiocarbonate',
 'J': 'Cyanomethyl methyl(phenyl)carbamodithioate',
 'DMF': 'Dimethylformamide',
 'Tol': 'Toluene'

In [60]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with all entries attributing to a sample analysis taken, described with a column of the right time

# ToDo: implement Mn too, and the different columns for the reaction substances: Monomer, Raft-Agent, Solvent
#    maybe also just do that later in the kinetic_curves_df

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in sRt.df.iterrows():

    kinetic_curve_entries = pd.DataFrame(index=range(len(polymerisation_kinetic[conversion_list])),
        data={"time" : hours_list, "conversion" : polymerisation_kinetic[conversion_list].values,
         "Mn" : polymerisation_kinetic[Mn_list].values, "Mw" : polymerisation_kinetic[Mw_list].values,
         # , "reactor" : polymerisation_kinetic["reactor"] reactor is not needed since the time is corrected
         })

    kinetic_curve_entries["exp_nr"] = str(index)
    kinetic_curve_entries["monomer"] = reaction_descriptors_dict[polymerisation_kinetic["monomer"]]
    kinetic_curve_entries["RAFT-Agent"] = reaction_descriptors_dict[polymerisation_kinetic["RAFT-Agent"]]
    kinetic_curve_entries["solvent"] = reaction_descriptors_dict[polymerisation_kinetic["solvent"]]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)


In [61]:
# reset Nans of Mn_t0 and Mw_t0 to 0 where the values are NaN
for kinetic_curve in kinetic_curves:
    kinetic_curve["Mn"].fillna(0, inplace=True)
    kinetic_curve["Mw"].fillna(0, inplace=True)


In [62]:
 # manually exclude experiment 301 as it disturbs the covariance plotting and has an unreasonable conversion value course.
if kinetic_curves[301]["exp_nr"].iloc[1] == "301":
    print(kinetic_curves[301])
    fig = px.line(kinetic_curves[301].dropna(subset="conversion"), x="time", y="conversion", title=f"kinetic 301 ", labels={"x":"time", "y":"conversion"})
    fig.add_scatter(x=kinetic_curves[301]["time"], y=kinetic_curves[301]["conversion"], mode="lines+markers", line=dict(dash="dot"), name="Mn")
    fig.show()
    kinetic_curves.pop(301)
else:
    print("Cell was already run. Experiment is deleted.")

        time conversion     Mn     Mw exp_nr          monomer  \
0   0.000000          0    0.0    0.0    301  4-Methylstyrene   
1   1.251389    0.02518  225.0  266.0    301  4-Methylstyrene   
2   2.241667   0.014388  234.0  265.0    301  4-Methylstyrene   
3   4.273611   0.014388  280.0  330.0    301  4-Methylstyrene   
4   6.237500        NaN  321.0  368.0    301  4-Methylstyrene   
5   8.338889   0.021583  352.0  416.0    301  4-Methylstyrene   
6  10.306944        NaN  357.0  429.0    301  4-Methylstyrene   
7  15.381944   0.032374  491.0  680.0    301  4-Methylstyrene   

                     RAFT-Agent            solvent  
0  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
1  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
2  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
3  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
4  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
5  2-Cyan-2-propylbenzodithioat  Dimethylformamide  
6  2-Cyan-2-propylbenzodithioat  Dimethylfo

In [63]:
'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)

def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)


'''
def neg_growth(x, l, k):
    y = l * (1 - np.exp(k * (-x)))
    return y

def neg_growth_derivative(x, l, k):
    y = l * k * np.exp(k*(-x))
    return y

# as the Mn values do not all start at 0
def neg_growth_abscissae(x, l, k, b):
    y = l * (1 - np.exp(k * (-x))) + b
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(m):
    return m

'''
xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
example_fig.show()
# '''
print("functions loaded")

functions loaded


In [154]:
# checking out max and min of Mn/Mw
min_Mn, max_Mn = [], []
min_Mw, max_Mw = [], []

for kc in kinetic_curves:
    min_Mn.append(kc["Mn"].min()/100000); max_Mn.append(kc["Mn"].max()/100000); min_Mw.append(kc["Mw"].min()/100000); max_Mw.append(kc["Mw"].max()/100000)

molar_wg_comparison = px.line()
molar_wg_comparison.add_scatter(y=min_Mn, name="min Mn", line_color="firebrick", opacity=0.8, fill="tozeroy", fillcolor="red")
molar_wg_comparison.add_scatter(y=max_Mn, name="max Mn", line_color="firebrick", opacity=0.8)
molar_wg_comparison.add_scatter(y=min_Mw, name="min Mw", line_color="darkgoldenrod", opacity=0.8, fill="tozeroy", fillcolor="orange")
molar_wg_comparison.add_scatter(y=max_Mw, name="max Mw", line_color="darkgoldenrod", opacity=0.8)
molar_wg_comparison.update_layout(title="Mn and Mw comparison", yaxis_title="$\\text{Mn and Mw }[g/mol] \cdot 10^{-5}$")
molar_wg_comparison.show()

In [102]:
fig = px.line(title="Kinetic Curve Fit")
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100) # x data array for plotting the fits

kinetics_df = pd.DataFrame() # create new dataframe with kinetics per row

def add_fits_to_plot(fit_func, fit_func_derivative, fit_func_params, marker_dic):
    fig.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", marker=marker_dic)
    fig.add_scatter(
        x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
        opacity=0.3, line=dict(dash="dash"), name=f"{fit_func_derivative.__name__}", marker=marker_dic)

def fit_and_exclude_outliers(x, y, fit_func, p0, bounds, nan_policy="omit", iteration=1, outliers=None):
    outliers = outliers if outliers is not None else []

    # exclude the nan values from the data
    mask = ~np.isnan(y)
    x, y = x[mask], y[mask]

    cf_data = curve_fit(f=fit_func, xdata=x, ydata=y, p0=p0, nan_policy=nan_policy, maxfev=800 * 10, bounds=bounds)

    # calculate the fit points
    fit_points = np.array([fit_func(x, *cf_data[0]) for x in x])
    # calculate the standard deviation of the residuals between the fit and the data points
    sigma = np.std(fit_points - y)

    # exclude the outliers
    msk = ~(np.abs(fit_points - y) > 2 * sigma)
    if not msk.all():
        return fit_and_exclude_outliers(x=x[msk], y=y[msk], fit_func=fit_func, p0=cf_data[0], bounds=bounds, nan_policy=nan_policy, iteration=iteration+1, outliers=outliers + [x[~msk]])

    # calculate the squared error of fit and data points
    sq_err = np.mean(np.square(y - fit_points))

    result = {"x": x, "y": y, "p_opt" : cf_data[0], "p_cov" : cf_data[1], "sq_err": sq_err, "excluded_points" : (x[~msk],y[~msk]), "iteration" : iteration, "outliers" : outliers}
    return result

# ToDo: this is exclusively for conversion yet, adding Mn, Mw support later
for idx, kinetic_curve in enumerate(kinetic_curves):
    # first make sure the datapoints are in the right format and not sometimes int sometimes float
    ydata_conv = np.array(kinetic_curve["conversion"].values, dtype=float)
    ydata_Mn = np.array(kinetic_curve["Mn"].values, dtype=float)/100000 # make it more comparable to conversion
    ydata_Mw = np.array(kinetic_curve["Mw"].values, dtype=float)/100000
    xdata = np.array(kinetic_curve["time"].values, dtype=float)


    # fitting section for conversion
    p_initial = [max(ydata_conv), 0.1] # this is a mandatory initial guess
    ng_fit = fit_and_exclude_outliers(x=xdata, y=ydata_conv, fit_func=neg_growth, p0=p_initial, bounds=([0, -np.inf], [1, np.inf]))
    # l_fit = fit_and_exclude_outliers(x=xdata, y=ydata, fit_func=linear_growth, p0=[max(ydata)/7], bounds=([0], [np.inf]))

    popt, pcov = ng_fit["p_opt"], ng_fit["p_cov"]
    conv_time_data = np.array([ng_fit["y"], ng_fit["x"]])
    squared_error = ng_fit["sq_err"]

    # fitting section for Mn
    p_initial = [max(ydata_Mn), 0.1, 0]
    ng_fit_Mn = fit_and_exclude_outliers(x=xdata, y=ydata_Mn, fit_func=neg_growth_abscissae, p0=p_initial, bounds=([0, -np.inf, -np.inf], [1, np.inf, np.inf]))

    popt_Mn, pcov_Mn = ng_fit_Mn["p_opt"], ng_fit_Mn["p_cov"]
    Mn_time_data = np.array([ng_fit_Mn["y"], ng_fit_Mn["x"]])
    squared_error_Mn = ng_fit_Mn["sq_err"]

    # fitting section for Mw
    ng_fit_Mw = fit_and_exclude_outliers(x=xdata, y=ydata_Mw, fit_func=neg_growth_abscissae, p0=p_initial, bounds=([0, -np.inf, -np.inf], [1, np.inf, np.inf]))

    popt_Mw, pcov_Mw = ng_fit_Mw["p_opt"], ng_fit_Mw["p_cov"]
    Mw_time_data = np.array([ng_fit_Mw["y"], ng_fit_Mw["x"]])
    squared_error_Mw = ng_fit_Mw["sq_err"]


    new_row = pd.DataFrame({"exp_nr":kinetic_curve["exp_nr"].iloc[1], "max_con":max(ydata_conv),
                        "theo_max_con":"yet to calc", "theo_react_end":"yet to calc",
                            "monomer":kinetic_curve["monomer"].iloc[1], "RAFT-Agent":kinetic_curve["RAFT-Agent"].iloc[1],
                            "solvent":kinetic_curve["solvent"].iloc[1],
                            "fit_p1":[popt[0]],"fit_p2":[popt[1]],
                            "p1_variance":[pcov[0][0]],"p1_p2_covariance":[pcov[0][1]],"p2_variance":[pcov[1][1]],
                            "squared_error":squared_error, "conv/time_data":conv_time_data})
    kinetics_df = pd.concat([kinetics_df, new_row])

''' comment out here if plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["exp_nr"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, marker_dict)

fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),

)
fig.show()
# '''

kinetics_df.reset_index(drop=True, inplace=True)
kinetics_df.drop(axis="index", index=kinetics_df[kinetics_df["max_con"] <= 0].index, inplace=True)
kinetics_df.reset_index(drop=True, inplace=True)
kinetics_df

ValueError: `x0` is infeasible.

In [76]:
# normalize the errors by dividing them by their respective standard deviation
def normalize_errors(err):
    return err / np.std(err)

for error in ["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]:
    kinetics_df[error] = normalize_errors(kinetics_df[error])

In [77]:
# kinetics_df[kinetics_df["squared_error"]>0.01] 29/317 entries
# calculating covariance between errors to use only the reasonable ones

# permutate all combinations of the errors
errorcombs = [[],[],[]]
for err1, err2 in itertools.combinations(["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"], 2):
    # print(f"The covariance between {err1} and {err2} is {np.cov(kinetics_df[err1], kinetics_df[err2])}")
    # print(f"The correlation between {err1} and {err2} is {np.correlate(kinetics_df[err1], kinetics_df[err2])}")
    # print("\n")
    errorcombs[0].append(f"{err1}/{err2}")
    errorcombs[1].append(np.cov(kinetics_df[err1], kinetics_df[err2]))
    errorcombs[2].append(*np.correlate(kinetics_df[err1], kinetics_df[err2]))
errorcombs_dc = {"name": errorcombs[0], "covariance":errorcombs[1], "correlation":errorcombs[2]}
errorcombs_df = pd.DataFrame(data=errorcombs_dc)
errorcombs_df["i_correlation"] = errorcombs_df["correlation"] * (-1)

# plot bar plot per a pair of errors with superimposed correlation
err_fig = px.bar(title="Correlation between Errors", labels={"correlation":"correlation"}, log_y=True)
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["correlation"], name="positive correlation", marker_color="green")
err_fig.add_bar(x=errorcombs_df["name"], y=errorcombs_df["i_correlation"], name="negative correlation", marker_color="crimson")
err_fig.show()

In [67]:
# violin plots of the errors
for errors in ["p1_variance", "p2_variance", "p1_p2_covariance"]:
    kinetics_df[errors] = kinetics_df[errors].apply(lambda x: np.absolute(x))
fig = px.violin(kinetics_df, y=["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"], box=True, points="all", log_y=True)
fig.update_layout(title="Errors normalize by \u03C3", xaxis_title="Error type", yaxis_title="value", yaxis=dict(title="log(value)", range=(-22, 3)))
fig.show()

In [68]:
# get information about within which error margin the fits and therefore kinetics are good
# split error in quartiles, index per error

title_dic = {"squared_error":"r² - error",
             "p1_variance":"variance of parameter 1",
             "p2_variance":"variance of parameter 2",
             "p1_p2_covariance":"covariance (Params 1 & 2)"}

def get_quartile_indexes(error_type):
    quartile_len = len(kinetics_df) / 4
    quartile_ranges = np.array([(a * quartile_len, (a + 1) * quartile_len) for a in range(4)], dtype=int)
    quartiles_list = list()
    for q in range(4):
        quartiles_list.append(kinetics_df.sort_values(by=[error_type]).iloc[quartile_ranges[q][0]:quartile_ranges[q][1]])
    return quartiles_list

def export_quartile_figures():
    for err in tqdm(["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]):
        err_quartiles = get_quartile_indexes(err)

        # plot the single quartiles
        for nr, n in enumerate(err_quartiles):
            iks = n.index
            fg = px.line(title=f"kinetics (quartile {nr+1} of the {title_dic[err]}) ", labels={"x":"time", "y":"conversion"})
            for ik in iks:
                marker_dict = dict(color=colors[ik%len(colors)])
                fit_data = kinetics_df.iloc[ik]
                xdt, ydt = fit_data["xdata"], fit_data["ydata"]
                fg.add_scatter(x=xdt, y=ydt, mode="lines+markers", name=kinetic_curves[ik]["exp_nr"].iloc[0], marker=marker_dict)
                add_fits_to_plot(neg_growth, neg_growth_derivative, [fit_data["fit_p1"], fit_data["fit_p2"]], marker_dict)
                fg.update_layout(yaxis=dict(range=[-0.1,1]), xaxis_title="time [h]", yaxis_title="conversion [%]")
            fg.write_image(f"data exploration figures/{err} ({nr+1} quartile).svg")
    print("exported")
# export_quartile_figures()

In [69]:
# descry when a function aligns to the datapoints in a reasonable way
#   Hence, wheneth' the blunder exaggerates, an 80% betweeneth' of the maximum conversion in that kinetic should be assessed to be the maximum conversion.
#   let's give a point for every quartile further from the first for the single errors divided by the maximum score (that is 3*4=12)
error_list = ["squared_error", "p1_variance", "p2_variance", "p1_p2_covariance"]
error_dic = {}
score = np.zeros(len(kinetics_df), int)
for error in error_list:
    quartiles = get_quartile_indexes(error)
    # for every error we want to give an error per index
    for sc, quartile in enumerate(quartiles):
        if sc == 0:
            continue
        # each quartile is a dataframe, where the latter one raise a higher error ( 0, 1, 2, 3)
        for num in quartile.index:
            score[num]+=sc
kinetics_df["error_score"]=score

In [70]:
p1_values = kinetics_df["fit_p1"].values
average_conversion = p1_values.mean()
min_con, max_con = p1_values.min(), p1_values.max()

print(f"According to the fits the mean maximum conversion is {average_conversion:.3n}, the minimum maximum is {min_con:.3n} and the maximum overall is {max_con:.3n}")

According to the fits the mean maximum conversion is 0.779, the minimum maximum is 0.0191 and the maximum overall is 1


In [71]:
# determining apposite reaction end values.
# The moment where 90% of the maximum conversion have been reached can be seen as a practical maximum conversion point
kinetics_df["theo_max_con"] = kinetics_df["fit_p1"].apply(lambda x: x*0.9)

def y_converted_negative_growth(y, l, k):
    return -np.log(1 - y / l) / k

kinetics_df["theo_react_end"] = \
    [y_converted_negative_growth(y, fit_p1, fit_p2) for y, fit_p1, fit_p2 in zip(kinetics_df["theo_max_con"], kinetics_df["fit_p1"], kinetics_df["fit_p2"])]

# the theoretical maximal conversion must be capped at reasonable time (we take two days here) that is applying 139/313 entries
kinetics_df["theo_react_end"] = [72 if x > 72 else x for x in kinetics_df["theo_react_end"].values]
# recalculate the apposite maximal conversion
kinetics_df["theo_max_con"] = [neg_growth(x, p1, p2) for x, p1, p2 in zip(kinetics_df["theo_react_end"], kinetics_df["fit_p1"], kinetics_df["fit_p2"])]
kinetics_df

,exp_nr,max_con,theo_max_con,theo_react_end,monomer,RAFT-Agent,solvent,fit_p1,fit_p2,p1_variance,p1_p2_covariance,p2_variance,squared_error,xdata,ydata,error_score
0,0,0.516574,0.698886,31.870087,Methyl methacrylate,2-Cyan-2-propylbenzodithioat,Dimethylformamide,0.776540,0.072249,1.309556e-06,5.279765e-05,0.000444,0.024205,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.10436494913029215, 0.2057761732851986,...",3
1,1,0.697321,0.733482,17.964494,Methyl methacrylate,2-Cyan-2-propylbenzodithioat,Dimethylsulfoxide,0.814980,0.128174,2.560277e-08,2.138887e-06,0.000041,0.003309,"[0.0, 2.011111111111111, 4.018055555555556, 8....","[0.0, 0.18921601058551107, 0.327820046311611, ...",0
2,2,0.323281,0.843708,72.000000,Methyl methacrylate,2-Cyan-2-propylbenzodithioat,Toluene,1.000000,0.025778,1.950291e-06,1.700630e-05,0.000030,0.000521,"[0.0, 4.0777777777777775, 8.090277777777779, 1...","[0.0, 0.09824902723735407, 0.1877431906614786,...",1
3,3,0.565125,0.857779,38.418587,Butyl methacrylate,2-Cyan-2-propylbenzodithioat,Dimethylformamide,0.953088,0.059934,4.250427e-06,1.087931e-04,0.000574,0.046853,"[0.0, 1.073611111111111, 2.0680555555555555, 4...","[0.0, 0.06908094948265364, 0.09403530127814974...",3
4,4,0.790565,0.900000,23.574040,Butyl methacrylate,2-Cyan-2-propylbenzodithioat,Dimethylsulfoxide,1.000000,0.097675,1.867054e-06,9.323417e-05,0.000999,0.084355,"[0.0, 1.1319444444444444, 2.125, 8.18055555555...","[0.0, 0.11448317307692302, 0.18088942307692302...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,312,0.734328,0.716837,13.932218,(2-Dimethylaminoethyl)methacrylate,2-(Dodecylthiocarbonothioylthio)-2-methylpropi...,Dimethylformamide,0.796486,0.165271,4.067339e-09,4.711542e-07,0.000014,0.001351,"[0.0, 1.2513888888888889, 2.2416666666666667, ...","[0.0, 0.14925373134328357, 0.24477611940298516...",0
309,313,0.546547,0.836712,11.266675,(2-Dimethylaminoethyl)methacrylate,2-(Dodecylthiocarbonothioylthio)-2-methylpropi...,Dimethylsulfoxide,0.929680,0.204371,1.148956e-08,1.032885e-06,0.000019,0.000074,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.21921921921921916, 0.3483483483483484,...",0
310,314,0.360000,0.344913,11.648310,(2-Dimethylaminoethyl)methacrylate,2-(Dodecylthiocarbonothioylthio)-2-methylpropi...,Toluene,0.383237,0.197675,2.602230e-07,7.719540e-05,0.006376,0.132912,"[0.0, 1.3125, 2.301388888888889, 4.33611111111...","[0.0, 0.11142857142857143, 0.11714285714285722...",5
311,315,0.840426,0.836045,14.901526,Benzyl methacrylate,2-(Dodecylthiocarbonothioylthio)-2-methylpropi...,Dimethylformamide,0.928939,0.154520,6.427724e-07,5.876434e-05,0.001335,0.186720,"[0.0, 1.3736111111111111, 2.3694444444444445, ...","[0.0, 0.14893617021276595, 0.2695035460992907,...",5


In [72]:
# doing the same fitting for Mn and investigating the discrepancy.
sRt.df

,Experiment number,experiment determiner,monomer,RAFT-Agent,solvent,sample determiner,reactor,t0h-Mn,t0h-Mw,t0h-Ð,...,reactor is underfilled after polymerization?,solution is decolorized?,solution in reactor is cloudy?,Precipitate in reactor?,hood on top of reactor? (phase separation),content of reactor gelated (bulk not hood or precipitate)?,use data for AI,possible sample determiner-original,t6h-conversion,t10h-conversion
0,MRG-046-G,B,6,A,DMF,MRG-046-G-B-6-A-DMF,1,NaN,NaN,NaN,...,0,3,0,0,0,0,NaN,6-A-DMF,NaN,NaN
1,MRG-046-G,B,6,A,DMSO,MRG-046-G-B-6-A-DMSO,2,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,6-A-DMSO,NaN,NaN
2,MRG-046-G,B,6,A,Tol,MRG-046-G-B-6-A-Tol,3,NaN,NaN,NaN,...,1,0,0,0,0,0,NaN,6-A-Tol,NaN,NaN
3,MRG-046-G,B,7,A,DMF,MRG-046-G-B-7-A-DMF,4,NaN,NaN,NaN,...,0,3,0,0,0,0,NaN,7-A-DMF,NaN,NaN
4,MRG-046-G,B,7,A,DMSO,MRG-046-G-B-7-A-DMSO,5,NaN,NaN,NaN,...,0,0,1,0,1,0,NaN,7-A-DMSO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,MRG-046-G,ZJ,9,E,DMF,MRG-046-G-ZJ-9-E-DMF,10,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,9-E-DMF,NaN,NaN
313,MRG-046-G,ZJ,9,E,DMSO,MRG-046-G-ZJ-9-E-DMSO,11,NaN,NaN,NaN,...,0,0,1,0,1,0 (hood = 1),NaN,9-E-DMSO,NaN,NaN
314,MRG-046-G,ZJ,9,E,Tol,MRG-046-G-ZJ-9-E-Tol,12,NaN,NaN,NaN,...,1,0,0,0,0,0,NaN,9-E-Tol,NaN,NaN
315,MRG-046-G,ZJ,10,E,DMF,MRG-046-G-ZJ-10-E-DMF,13,NaN,NaN,NaN,...,0,0,0,0,0,0,NaN,10-E-DMF,NaN,NaN
